In [1]:
#importing dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score,roc_auc_score
import sklearn
import numpy as np
import pandas as pd
import plotly as plot

from sklearn.metrics import accuracy_score
import plotly.offline as pyo
from plotly.offline import init_notebook_mode,plot,iplot
dataset=pd.read_csv('cleve.csv')

X=dataset.iloc[:,:-1].values
y=dataset.iloc[:,13].values
dataset.head()
#imputation for data cleaning i.e; replace missing values with mean values
from sklearn.impute import SimpleImputer
Imputation=SimpleImputer(strategy="mean",missing_values=np.nan, fill_value=None, verbose=0, copy=True, add_indicator=False)
X[:,11:13]=Imputation.fit_transform(X[:,11:13])

#divide dataset into train and test set
from sklearn.model_selection import train_test_split,GridSearchCV
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=63)
#Scaling is a technique of feature Scaling process
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler(with_mean=True,with_std=True)
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
X_train, X_train_lr, y_train, y_train_lr = train_test_split(X_train, y_train, test_size=0.2,random_state=43)
import xgboost as xgb
xgb = xgb.XGBClassifier(nthread=4,     #Meaning: When nthread=-1, use all CPUs for parallel operation (default), when nthread=1, use 1 CPU for operation.
                          learning_rate=0.001,    #Meaning: learning rate, which controls the step size when the weight is updated in each iteration, the default is 0.3. Tuning: The smaller the value, the slower the training. The typical value is 0.01-0.2.
                          n_estimators=50,       #Meaning: The total number of iterations, that is, the number of decision trees
                          max_depth=5,         #Meaning: the depth of the tree, the default value is 6, the typical value is 3-10. Parameter tuning: the larger the value, the easier it is to overfit; the smaller the value, the easier it is to underfit
                          gamma=0,               #Meaning: The penalty term coefficient specifies the minimum loss function drop value required for node splitting.
                          subsample=0.9,       #Meaning: When training each tree, the proportion of data used in the total training set. The default value is 1, and the typical value is 0.5-1. Tuning: Prevent overfitting.
                          colsample_bytree=0.5,use_label_encoder=False) #When training each tree, the proportion of features used in total features. The default value is 1, and the typical value is 0.5-1. Tuning: Prevent overfitting.
 
xgb_enc = OneHotEncoder()
xgb_lm = LogisticRegression(solver='lbfgs', max_iter=10000)
xgb.fit(X_train, y_train)
xgb_enc.fit(xgb.apply(X_train))
xgb_lm.fit(xgb_enc.transform(xgb.apply(X_train_lr)), y_train_lr) 
y_pred_xgb_lm = xgb_lm.predict_proba(xgb_enc.transform(xgb.apply(X_test)))[:, 1]

#fpr_xgb_lm, tpr_xgb_lm, _ = roc_curve(y_test, y_pred_xgb_lm)
#print("The AUC of xgboost+LR is:", roc_auc_score(y_test, y_pred_xgb_lm))
l=[round(i) for i in y_pred_xgb_lm]
print(accuracy_score(y_test,l)*100)
#print(mean_squared_error(y_test,y_pred_xgb_lm),r2_score(y_test,y_pred_xgb_lm))

[18:14:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
83.60655737704919
